In [2]:
import edgar
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bsoup
import numpy as np
import string
from cleanMDA import extractTable, getComp, divide_chunks, annual_filings, MDA

# Get 2007-2012 Companies

In [3]:
res = requests.get('https://etfdb.com/history-of-the-s-and-p-500/#2009')
soup = bsoup(res.content,'html.parser')
tbl = extractTable(soup)

In [4]:
# Remove companies 1980-1999
Lst = tbl[200:]
# Split into chunks of 10 CO per year 
tbl = list(divide_chunks(Lst, 10))
colNames=[i for i in range(2000,2014)]
# Make df
dfTop10 = pd.DataFrame(tbl).transpose()
dfTop10.columns = colNames

In [5]:
# Remove punctuation and uppercase 
for i in colNames:
    dfTop10[i] = dfTop10[i].str.upper().str.strip(string.punctuation).str.strip('\n').str.strip()


In [6]:
dfTop10.replace({'CHEVRON CORP': 'CHEVRON', 'CHEVRON': 'CHEVRON CORP','MICROSOFT': 'MICROSOFT CORP','GENERAL ELECTRIC': 'GENERAL ELECTRIC CO', 'EXXON MOBIL':'EXXON MOBIL CORP', 'PFIZER':'PFIZER INC', 'CITIGROUP':'CITIGROUP INC', 'CISCO SYSTEMS':'CISCO SYSTEMS INC', 'WAL-MART STORES' :'WALMART INC','AIG':'AMERICAN INTERNATIONAL GROUP','MERCK':'MERCK & CO','INTEL':'INTEL CORP','PROCTER & GAMBLE':'PROCTER & GAMBLE CO','AT&T':'AT&T CORP','JPMORGAN CHASE':'JPMORGAN CHASE & CO','GOOGLE':'ALPHABET INC','APPLE':'APPLE INC','BANK OF AMERICA':'BANK OF AMERICA CORP','IBM':'INTL BUSINESS MACHINES CORP','CHEVRON':'CHEVRON CORP','WELLS FARGO':'WELLS FARGO & CO'}, regex=True, inplace=True)

In [7]:
dfCik = pd.read_csv('2000_2013_s&p.csv')
dfCik.drop(['gvkey','gvkeyx','from','thru','tic'], axis=1, inplace=True)
dfCik['co_cik'] = dfCik['co_cik'].fillna(0).astype(int).apply(lambda x: '{0:0>10}'.format(x))
cikDic = pd.Series(dfCik.co_conm.values,index=dfCik.co_cik).to_dict()
cikDic = dict((v,k) for k,v in cikDic.items())

In [8]:
from collections import defaultdict
dic = defaultdict(None)
# Create a dictionary of Company Name and CIK #
for yr in colNames:  
    for comp in dfTop10[yr]:
        if comp in cikDic:
            dic[comp] = cikDic[comp]
        else:
            dic[comp] = None
        

In [11]:
# Create Company Name, CIK number tuple pairs 
for yr in colNames:  
    
    for i in range(len(dfTop10[yr])):
        if type(dfTop10[yr][i])==tuple:
            pass
        else:
            name = dfTop10[yr][i]
            dfTop10[yr][i] =  (name,dic[name])

In [13]:
dfTop10 = dfTop10.drop(columns=[2000,2001,2002,2003,2004,2005,2006,2013])

In [14]:
# Clean wrong CIK numbers 
for year in dfTop10.columns:
    for cmp in range(10):
        if dfTop10[year][cmp] == ('ALPHABET INC', '0001652044'):
            dfTop10[year].loc[cmp] = ('ALPHABET INC', '0001288776')
        elif dfTop10[year][cmp] == ('AT&T CORP', '0000005907'):
            dfTop10[year].loc[cmp] = ('AT&T CORP', '0000732717')

In [15]:
# reorder the yeara
dfTop10 = dfTop10[dfTop10.columns[::-1]]

# Get 2013-2017 Companies

In [16]:
# read in top 100 2013-2017 companies
df = pd.read_html('http://siblisresearch.com/data/market-caps-sp-100-us/')[0]

In [17]:
# get top 10 companies from each year
df_2017 = pd.DataFrame(df['Company Name'])
df_2017['12/31/2017'] = df['12/31/2017']
df_2017 = df_2017.sort_values(by=['12/31/2017'], ascending=False)
top_2017 = pd.DataFrame(np.array(df_2017.head(10)['Company Name']), columns=['2017'])

df_2016 = pd.DataFrame(df['Company Name'])
df_2016['12/31/2016'] = df['12/31/2016']
df_2016 = df_2016.sort_values(by=['12/31/2016'], ascending=False)
top_2016 = pd.DataFrame(np.array(df_2016.head(10)['Company Name']), columns=['2016'])

df_2015 = pd.DataFrame(df['Company Name'])
df_2015['12/31/2015'] = df['12/31/2015']
df_2015 = df_2015.sort_values(by=['12/31/2015'], ascending=False)
top_2015 = pd.DataFrame(np.array(df_2015.head(10)['Company Name']), columns=['2015'])

df_2014 = pd.DataFrame(df['Company Name'])
df_2014['12/31/2014'] = df['12/31/2014']
df_2014 = df_2014.sort_values(by=['12/31/2014'], ascending=False)
top_2014 = pd.DataFrame(np.array(df_2014.head(10)['Company Name']), columns=['2014'])

df_2013 = pd.DataFrame(df['Company Name'])
df_2013['12/31/2013'] = df['12/31/2013']
df_2013 = df_2013.sort_values(by=['12/31/2013'], ascending=False)
top_2013 = pd.DataFrame(np.array(df_2013.head(10)['Company Name']), columns=['2013'])

In [18]:
top10_2017 = pd.concat([top_2017,top_2016,top_2015,top_2014,top_2013],axis=1)

In [20]:
# get an array of the Company names and CIK numbers
companies = pd.DataFrame.from_csv('companies_2017.csv',encoding='latin-1')
companies = companies[:505]
companies.CIK = companies['CIK'].astype(int).apply(lambda x: str(x).zfill(10))

name = [i.strip(string.punctuation) for i in np.array(companies['Security'])]
num = np.array(companies['CIK'])

# get companies CIK numbers
CIK_num = dict()
for i in range(len(name)):
    CIK_num[name[i]] = num[i]

# manually add in ID for Company Name's missed
CIK_man = {'Alphabet Inc. Class A':'0001652044','Berkshire Hathaway Inc':'0001067983', 'Wal-Mart Stores Inc':'0000104169', 
           'IBM Corp.':'0000051143', 'The Coca Cola Co.':'0000021344', "Philip Morris Int'l":'0001413329', 
           'Facebook Inc.':'0001326801', 'The Walt Disney Co.':'0001001039', 'Comcast Class A Comm.':'0001166691'}

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  


In [21]:
# returns an array of each companies CIK number
for year in top10_2017:
    for i in range(len(top10_2017[year])):
        if top10_2017[year][i].strip(string.punctuation) in CIK_num:
            top10_2017[year][i] = (top10_2017[year][i],CIK_num[top10_2017[year][i].strip(string.punctuation)])
        elif int(year) < 2016 and top10_2017[year][i] == 'Alphabet Inc. Class A':
            top10_2017[year][i] = (top10_2017[year][i],'0001288776')
        elif top10_2017[year][i] in CIK_man:
            top10_2017[year][i] = (top10_2017[year][i],CIK_man[top10_2017[year][i]])
        else:
            top10_2017[year][i] = (top10_2017[year][i], None)
top10_2017.head()

,2017,2016,2015,2014,2013
0,"(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)"
1,"(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001288776)","(Exxon Mobil Corp., 0000034088)","(Exxon Mobil Corp., 0000034088)"
2,"(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Alphabet Inc. Class A, 0001288776)"
3,"(Amazon.com Inc., 0001018724)","(Berkshire Hathaway Inc, 0001067983)","(Berkshire Hathaway Inc, 0001067983)","(Berkshire Hathaway Inc, 0001067983)","(Microsoft Corp., 0000789019)"
4,"(Facebook Inc., 0001326801)","(Exxon Mobil Corp., 0000034088)","(Exxon Mobil Corp., 0000034088)","(Alphabet Inc. Class A, 0001288776)","(Berkshire Hathaway Inc, 0001067983)"


In [22]:
# combine all the years together
dfTop10 = pd.concat([top10_2017,dfTop10],axis=1)
# change all the years to integers
dfTop10 = pd.DataFrame(np.array(dfTop10), columns=dfTop10.columns.astype(int))
dfTop10.head(3)

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(APPLE INC, 0000320193)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)"
1,"(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001288776)","(Exxon Mobil Corp., 0000034088)","(Exxon Mobil Corp., 0000034088)","(EXXON MOBIL CORP, 0000034088)","(APPLE INC, 0000320193)","(APPLE INC, 0000320193)","(MICROSOFT CORP, 0000789019)","(WALMART INC, 0000104169)","(GENERAL ELECTRIC CO, 0000040545)"
2,"(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Alphabet Inc. Class A, 0001288776)","(MICROSOFT CORP, 0000789019)","(MICROSOFT CORP, 0000789019)","(MICROSOFT CORP, 0000789019)","(WALMART INC, 0000104169)","(PROCTER & GAMBLE CO, 0000080424)","(MICROSOFT CORP, 0000789019)"


# Find the indexes of each company's MDA text

In [23]:
# create a dataframe to hold the indexes and document number
dfTop10_index = dfTop10.copy()

In [24]:
dfTop10_index.head(3)

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(APPLE INC, 0000320193)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)"
1,"(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001288776)","(Exxon Mobil Corp., 0000034088)","(Exxon Mobil Corp., 0000034088)","(EXXON MOBIL CORP, 0000034088)","(APPLE INC, 0000320193)","(APPLE INC, 0000320193)","(MICROSOFT CORP, 0000789019)","(WALMART INC, 0000104169)","(GENERAL ELECTRIC CO, 0000040545)"
2,"(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Alphabet Inc. Class A, 0001288776)","(MICROSOFT CORP, 0000789019)","(MICROSOFT CORP, 0000789019)","(MICROSOFT CORP, 0000789019)","(WALMART INC, 0000104169)","(PROCTER & GAMBLE CO, 0000080424)","(MICROSOFT CORP, 0000789019)"


Note that the below code takes some time to run as it pulls the indices for MDA for all years. The final file is pickled below.

In [122]:
# go through each company and get the indices of which section to pull the MDA from
ct = 0
for year in dfTop10:
    ct += 1
    for i in range(len(dfTop10[year])):
        dfTop10_index[year].iloc[i] = annual_filings(dfTop10[year].iloc[i][0],dfTop10[year].iloc[i][1],year,ct)
dfTop10_index.head()

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., 0000320193, [10, 32, 41, 43], [14...","(Apple Inc., 0000320193, [10, 31, 40, 42], [14...","(Apple Inc., 0000320193, [10, 31, 40, 42], [14...","(Apple Inc., 0000320193, [10, 31, 40, 42], [14...","(Apple Inc., 0000320193, [10, 30, 39, 41, 42],...","(APPLE INC, 0000320193, [17, 19, 20], [36], 6)","(EXXON MOBIL CORP, 0000034088, [11, 35], [15, ...","(EXXON MOBIL CORP, 0000034088, [10, 33], [14, ...","(EXXON MOBIL CORP, 0000034088, [11, 35], [15, ...","(EXXON MOBIL CORP, 0000034088, [11, 34], [15, ...","(EXXON MOBIL CORP, 0000034088, [11, 35], [15, ..."
1,"(Alphabet Inc. Class A, 0001652044, [11, 26, 3...","(Alphabet Inc. Class A, 0001652044, [11, 25, 3...","(Alphabet Inc. Class A, 0001288776, [11, 25, 3...","(Exxon Mobil Corp., 0000034088, [11, 35, 52, 5...","(Exxon Mobil Corp., 0000034088, [11, 35, 52, 5...","(EXXON MOBIL CORP, 0000034088, [11, 35, 53, 54...","(APPLE INC, 0000320193, [9, 16, 18], [30], 7)","(APPLE INC, 0000320193, [9, 16, 18], [28], 8)","(MICROSOFT CORP, 0000789019, [10, 53], [14, 12...","(WALMART INC, 0000104169, [17], [24], 11)","(GENERAL ELECTRIC CO, 0000040545, [10, 34], [1..."
2,"(Microsoft Corp., 0000789019, [10, 67], [14, 1...","(Microsoft Corp., 0000789019, [10, 65], [14, 1...","(Microsoft Corp., 0000789019, [10, 64], [14, 1...","(Microsoft Corp., 0000789019, [10, 60], [14, 1...","(Alphabet Inc. Class A, 0001288776, [10, 24, 3...","(MICROSOFT CORP, 0000789019, [10, 57], [14, 12...","(MICROSOFT CORP, 0000789019, [9, 53], [13, 125...","(MICROSOFT CORP, 0000789019, [9, 55], [13, 126...","(WALMART INC, 0000104169, [17], [24], 10)","(PROCTER & GAMBLE CO, 0000080424, [10, 36], [1...","(MICROSOFT CORP, 0000789019, [10, 52], [14, 11..."
3,"(Amazon.com Inc., 0001018724, [10, 46], [14, 6...","(Berkshire Hathaway Inc, 0001067983, [10, 33, ...","(Berkshire Hathaway Inc, 0001067983, [10, 33, ...","(Berkshire Hathaway Inc, 0001067983, [10, 32, ...","(Microsoft Corp., 0000789019, [10, 56], [14, 1...","(GENERAL ELECTRIC CO, 0000040545, [10, 25, 27,...","(INTL BUSINESS MACHINES CORP, 0000051143, [14,...","(BERKSHIRE HATHAWAY, 0001067983, [10, 32, 34],...","(ALPHABET INC, 0001288776, [10, 33], [14, 45], 8)","(MICROSOFT CORP, 0000789019, [10, 52], [14, 11...","(AT&T CORP, 0000732717, [2, 13], [2, 20], 11)"
4,"(Facebook Inc., 0001326801, [10, 24, 33, 36], ...","(Exxon Mobil Corp., 0000034088, [11, 35, 53, 5...","(Exxon Mobil Corp., 0000034088, [11, 35, 53, 5...","(Alphabet Inc. Class A, 0001288776, [10, 24, 3...","(Berkshire Hathaway Inc, 0001067983, [10, 32, ...","(INTL BUSINESS MACHINES CORP, 0000051143, [14,...","(CHEVRON CORP, 0000093410, [2, 15], [2, 19], 7)","(GENERAL ELECTRIC CO, 0000040545, [10, 26, 28,...","(APPLE INC, 0000320193, [9, 17, 19], [30], 10)","(AT&T CORP, 0000732717, [2, 13], [2, 20], 10)","(PROCTER & GAMBLE CO, 0000080424, [10, 36], [1..."


# Find the MDA text for each company

In [25]:
# create a dataframe to hold the MDA text
dfTop10_MDA = dfTop10_index.copy()
dfTop10_MDA.head(3)

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(Apple Inc., 0000320193)","(APPLE INC, 0000320193)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)","(EXXON MOBIL CORP, 0000034088)"
1,"(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001652044)","(Alphabet Inc. Class A, 0001288776)","(Exxon Mobil Corp., 0000034088)","(Exxon Mobil Corp., 0000034088)","(EXXON MOBIL CORP, 0000034088)","(APPLE INC, 0000320193)","(APPLE INC, 0000320193)","(MICROSOFT CORP, 0000789019)","(WALMART INC, 0000104169)","(GENERAL ELECTRIC CO, 0000040545)"
2,"(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Microsoft Corp., 0000789019)","(Alphabet Inc. Class A, 0001288776)","(MICROSOFT CORP, 0000789019)","(MICROSOFT CORP, 0000789019)","(MICROSOFT CORP, 0000789019)","(WALMART INC, 0000104169)","(PROCTER & GAMBLE CO, 0000080424)","(MICROSOFT CORP, 0000789019)"


Again note that the following code will take time to run as this pulls the text given the indicies that were pulled in the code above. 

In [134]:
# go through each company and get the indices of which section to pull the MDA from
for year in dfTop10_index:
    for i in range(len(dfTop10_index[year])):
        if dfTop10_index[year][i][1] != None:
            dfTop10_MDA[year].iloc[i] = MDA(dfTop10_index[year].iloc[i][0],dfTop10_index[year].iloc[i][1],dfTop10_index[year].iloc[i][2],dfTop10_index[year].iloc[i][3],dfTop10_index[year].iloc[i][4],year)
        else:
            dfTop10_MDA[year].iloc[i]= dfTop10_index[year][i]

In [135]:
dfTop10_MDA.head()

,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
0,"(Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENT’S DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND AN...","(Apple Inc., [7 MANAGEMENTS DISCUSSION AND A...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND AN...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL ST...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIAL..."
1,"(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(EXXON MOBIL CORP, [15. EXHIBITS, FINANCIA...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSIO...","(WALMART INC, None)","(GENERAL ELECTRIC CO, None)"
2,"(Microsoft Corp., [7 MANAGEMENT’S DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION AN...","(WALMART INC, None)","(PROCTER & GAMBLE CO, None)","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSION..."
3,"(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(Microsoft Corp., [7 MANAGEMENTS DISCUSSION A...","(GENERAL ELECTRIC CO, [7 “MANAGEMENT’S DISCUSS...","(INTL BUSINESS MACHINES CORP, None)","(BERKSHIRE HATHAWAY, [ 7 MANAGEMENTS DISCUSSI...","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(MICROSOFT CORP, [7 MANAGEMENTS DISCUSSIO...","(AT&T CORP, None)"
4,"(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Exxon Mobil Corp., [15. EXHIBITS, FINANCI...","(Alphabet Inc. Class A, [7MANAGEMENT’S DISCUSS...","(Berkshire Hathaway Inc, [7 MANAGEMENTS DISCU...","(INTL BUSINESS MACHINES CORP, None)","(CHEVRON CORP, [15. EXHIBITS, ...","(GENERAL ELECTRIC CO, [7 MANAGEMENTS DISCUSS...","(APPLE INC, [7 MANAGEMENTS DISCUSSION AND ANA...","(AT&T CORP, None)","(PROCTER & GAMBLE CO, None)"
5,"(Berkshire Hathaway Inc, [7 MANAGEMENTS DISC...","(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...","(Amazon.com Inc., [7MANAGEMENT’S DISCUSSION AN...","(Johnson & Johnson, None)","(General Electric, [7 “MANAGEMENT’S DISCUSSION...","(CHEVRON CORP, [15. EXHIBITS, FINANCIAL STATEM...","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(WALMART INC, None)","(JOHNSON & JOHNSON, None)","(JOHNSON & JOHNSON, None)","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ..."
6,"(Johnson & Johnson, [7MANAGEMENT’S DISCUSSION ...","(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...","(Facebook Inc., [7MANAGEMENT'S DISCUSSION AND ...","(Wells Fargo, None)","(Johnson & Johnson, None)","(JOHNSON & JOHNSON, None)","(WALMART INC, None)","(ALPHABET INC, [7 MANAGEMENTS DISCUSSION AND ...","(PROCTER & GAMBLE CO, None)","(GENERAL ELECTRIC CO, [7 MANAGEMENT’S DISCUSSI...","(CHEVRON CORP, [15. EXHIBITS, ..."
7,"(JPMorgan Chase & Co., [601 OF REGULATION S-K....","(Johnson & Johnson, [7MANAGEMENT’S DISCUSSION ...","(General Electric, [ 22 MANAGEMENT'S DIS...","(Wal-Mart Stores Inc, None)","(Wal-Mart Stores Inc, None)","(AT&T CORP, None)","(GENERAL ELECTRIC CO, [7 “MANAGEMENT’S DISCUSS...","(CHEVRON CORP, [15. EXHIBITS, ...","(INTL BUSINESS MACHINES CORP, None)","(CHEVRON CORP, [15. EXHIBITS, ...","(JOHNSON & JOHNSON, None)"
8,"(Exxon Mobil Corp., [16. FO

In [136]:
dfTop10_MDA.to_pickle('Top10_MDA.pkl')